# Resolução de Redes em python

Giovanna de Freitas Velasco - 1367634
João Pedro Gomes - 

### Exercício A

In [ ]:
import numpy as np
#import matplotlib.pyplot as plt

In [ ]:
def GeraMatriz(nv, nc, C, conec):
    


In [1]:
import numpy as np
#import matplotlib.pyplot as plt

#declaração do número de linhas e colunas da grade
lin = 4
col = 5

#n == colunas
#m == linhas

#Função que cria uma Rede Hidraulica em grade quadrada
def CriaRedeQuad(lin, col, CH, CV):
    #numero de vertices e arestas
    nv = lin * col
    nc = (lin - 1) * col + (col - 1) * lin
    
    #criação da matriz de coordenadas dos vértices
    coord = np.zeros(shape = (nv,2))
    
    #enumeração dos vértices
    for j in range (lin):
        for i in range (col):
            ig = i + j * n
            coord[ig,0] = i
            coord[ig,1] = j
            
    #matriz de conexões e condutancia
    conec = np.zeros(shape = (nc,2), dtype = int)
    C = np.zeros(nc)
    
    #enumeração de canos horizontais
    for j in range (lin):
        for i in range (col - 1):
            k = j * (col - 1) + i
            conec[k,0] = j * col + i
            conec[k,1] = j * col + i + 1
            C[k] = CH
            
    #enumeração de canos verticais
    #for j in range ():
    
 